In [44]:
import pandas as pd
import sklearn
from sklearn.preprocessing import MultiLabelBinarizer
pd.set_option('display.max_columns', None)

## Import data

In [20]:
weapons = pd.read_csv('./data/Skyrim_Weapons.csv')
npcs = pd.read_csv('./data/Skyrim_Named_Characters.csv')
race_stats = pd.read_csv('./data/skyrimstats.csv')

The weapons data contains information on all of the weapons within Skyrim.

In [21]:
weapons.head()

,Name,Damage,Weight,Gold,Upgrade,Perk,Type,Category,Speed
0,Ancient Nord Battle Axe,18,22.0,28,Steel Ingot,Steel,Battleaxe,Two-Handed,NaN
1,Daedric Battleaxe,25,27.0,2750,Ebony Ingot,Daedric,Battleaxe,Two-Handed,NaN
2,Dragonbone Battleaxe,26,30.0,3000,Dragon Bone,Dragon,Battleaxe,Two-Handed,NaN
3,Drainblood Battleaxe,21,5.0,266,NaN,-,Battleaxe,Two-Handed,NaN
4,Dwarven Battleaxe,20,23.0,300,Dwarven Metal Ingot,Dwarven,Battleaxe,Two-Handed,NaN


The npcs dataframe contains information on the NPCs within Skyrim, such as their Race, abilities, Factions etc.

In [22]:
npcs.columns

Index(['Source', 'Name', 'Home City', 'House', 'Race', 'Gender', 'Level',
       'Class', 'RefID', 'BaseID', 'Health', 'Magicka', 'Stamina',
       'Primary Skills', 'Morality', 'Aggression', 'Faction(s)',
       'Class Details', 'Essential', 'Location', 'Training', 'Merchant',
       'Gold', 'Sells', 'Buys', 'Race Details', 'Follower', 'Respawns',
       'Store', 'AdoptHF', 'Protected', 'Perks', 'Marry', 'StewardHF',
       'Horses', 'Added by', 'Level (PC=10)', 'Health (PC=10)',
       'Magicka (PC=10)', 'Stamina (PC=10)'],
      dtype='object')

In [23]:
npcs.head()

,Source,Name,Home City,House,Race,Gender,Level,Class,RefID,BaseID,Health,Magicka,Stamina,Primary Skills,Morality,Aggression,Faction(s),Class Details,Essential,Location,Training,Merchant,Gold,Sells,Buys,Race Details,Follower,Respawns,Store,AdoptHF,Protected,Perks,Marry,StewardHF,Horses,Added by,Level (PC=10),Health (PC=10),Magicka (PC=10),Stamina (PC=10)
0,https://en.uesp.net/wiki/Skyrim:Beitild,Beitild,Dawnstar,Beitild's House,Nord,Female,6,Miner,0001A6B6,00013612,91,67,67,"Smithing, Two-handed",No Crime,Unaggressive,CrimeFactionPale; DawnstarIronBreakerMinersFac...,None,No,NaN,None,NaN,NaN,None,None,NaN,No,No,None,No,No,NaN,No,No,NaN,Vanilla,6.0,91.0,67.0,67.0
1,https://en.uesp.net/wiki/Skyrim:Brina_Merilis,Brina Merilis,Dawnstar,Brina's House,Nord,Female,PC×1 (range=10-25),Warrior,0001A6B8,0001A6B7,50+(PC-1)×11.7,50,50+(PC-1)×3.3,"Heavy Armor, One-handed, Archery, Block",No Crime,Unaggressive,CrimeFactionPale; DawnstarBrinaHouseFaction; F...,CombatWarrior1H,Yes,NaN,None,NaN,NaN,None,None,NaN,No,No,None,No,No,NaN,No,No,NaN,Vanilla,10.0,155.3,50.0,79.7
2,https://en.uesp.net/wiki/Skyrim:Horik_Halfhand,Horik Halfhand,Dawnstar,Brina's House,Nord,Male,PC×1 (range=10-25),Warrior,0001A6BA,0001A6B9,50+(PC-1)×11.7,50,50+(PC-1)×3.3,"Heavy Armor, One-handed, Archery, Block",No Crime,Unaggressive,CrimeFactionPale; DawnstarBrinaHouseFaction; I...,CombatWarrior1H,Yes,NaN,None,NaN,NaN,None,None,NaN,No,No,None,No,No,NaN,No,No,NaN,Vanilla,10.0,155.3,50.0,79.7
3,https://en.uesp.net/wiki/Skyrim:Dawnstar_Guard,Dawnstar Guard,NaN,NaN,Nord,Male,PCx1 (Range=20-50),Warrior,NaN,0002D794,NaN,NaN,NaN,NaN,Indeterminate,Indeterminate,CrimeFactionPale; Stormcloaks; Sons of Skyrim;...,None,No,Dawnstar,None,NaN,NaN,None,None,NaN,No,No,None,No,No,NaN,No,No,NaN,Vanilla,20.0,NaN,NaN,NaN
4,https://en.uesp.net/wiki/Skyrim:Babette,Babette,NaN,NaN,Breton,Female,10,Assassin,0001D4BC,0001D4B7,125,15,95,"Archery, One-handed, Sneak, Conjuration, Light...",No Crime,Unaggressive,Alchemy Trainer; Babette's Bed Ownership; Babe...,TrainerAlchemyMaster,Yes,"Dark Brotherhood Sanctuary, Dawnstar Sanctuary",Alchemy (Master),True,500 (+1000 Master Trader),See Standard Merchandise,"Animal Parts, Food, Ingredients, Poisons, Poti...","Vampire, Undead, Child",No,No,None,No,No,NaN,No,No,NaN,Vanilla,10.0,125.0,15.0,95.0


In [24]:
npcs.describe()

,Level (PC=10),Health (PC=10),Magicka (PC=10),Stamina (PC=10)
count,1009.000000,994.000000,992.000000,978.000000
mean,8.486125,138.280355,71.102092,81.599782
std,9.350765,144.704267,59.497145,62.055653
min,1.000000,0.000000,0.000000,0.000000
25%,4.000000,75.000000,50.000000,60.000000
50%,6.000000,91.000000,60.000000,67.000000
75%,10.000000,149.525000,67.000000,82.697500
max,150.000000,2071.000000,626.360000,1043.000000


The race_stats dataframe contains information on the base statistics each of the races in Skyrim.

In [25]:
race_stats.head(n=20)

,Unnamed: 0,Race,Smithing,Heavy Armor,Block,Two-Handed,One-Handed,Archery,Light Armor,Sneak,Lockpicking,Pickpocket,Speech,Alchemy,Illusion,Conjuration,Destruction,Restoration,Alteration,Enchanting
0,0,Altmer (High Elf),15,15,15,15,15,15,15,15,15,15,15,15,25,20,20,20,20,20
1,1,Argonian,15,15,15,15,15,15,20,20,25,20,15,15,15,15,15,20,20,15
2,2,Bosmer (Wood Elf),15,15,15,15,15,25,20,20,20,20,15,20,15,15,15,15,15,15
3,3,Breton,15,15,15,15,15,15,15,15,15,15,20,20,20,25,15,20,20,15
4,4,Dunmer (Dark Elf),15,15,15,15,15,15,20,20,15,15,15,20,20,15,25,15,20,15
5,5,Imperial,15,20,20,15,20,15,15,15,15,15,15,15,15,15,20,25,15,20
6,6,Khajiit,15,15,15,15,20,20,15,25,20,20,15,20,15,15,15,15,15,15
7,7,Nord,20,15,20,25,20,15,20,15,15,15,20,15,15,15,15,15,15,15
8,8,Orsimer (Orc),20,25,20,20,20,15,15,15,15,15,15,15,15,15,15,15,15,20
9,9,Redguard,20,15,20,15,25,20,15,15,15,15,15,15,15,15,20,15,20,15


## NPC analysis

In this section, I will investigate a few different questions.

- Can you predict the race of an NPC based on their faction, statistics etc?
- Are certain locations populated by more aggressive NPCs? This would seem likely based on gameplay!

### Data cleaning

In [26]:
npcs.head()

,Source,Name,Home City,House,Race,Gender,Level,Class,RefID,BaseID,Health,Magicka,Stamina,Primary Skills,Morality,Aggression,Faction(s),Class Details,Essential,Location,Training,Merchant,Gold,Sells,Buys,Race Details,Follower,Respawns,Store,AdoptHF,Protected,Perks,Marry,StewardHF,Horses,Added by,Level (PC=10),Health (PC=10),Magicka (PC=10),Stamina (PC=10)
0,https://en.uesp.net/wiki/Skyrim:Beitild,Beitild,Dawnstar,Beitild's House,Nord,Female,6,Miner,0001A6B6,00013612,91,67,67,"Smithing, Two-handed",No Crime,Unaggressive,CrimeFactionPale; DawnstarIronBreakerMinersFac...,None,No,NaN,None,NaN,NaN,None,None,NaN,No,No,None,No,No,NaN,No,No,NaN,Vanilla,6.0,91.0,67.0,67.0
1,https://en.uesp.net/wiki/Skyrim:Brina_Merilis,Brina Merilis,Dawnstar,Brina's House,Nord,Female,PC×1 (range=10-25),Warrior,0001A6B8,0001A6B7,50+(PC-1)×11.7,50,50+(PC-1)×3.3,"Heavy Armor, One-handed, Archery, Block",No Crime,Unaggressive,CrimeFactionPale; DawnstarBrinaHouseFaction; F...,CombatWarrior1H,Yes,NaN,None,NaN,NaN,None,None,NaN,No,No,None,No,No,NaN,No,No,NaN,Vanilla,10.0,155.3,50.0,79.7
2,https://en.uesp.net/wiki/Skyrim:Horik_Halfhand,Horik Halfhand,Dawnstar,Brina's House,Nord,Male,PC×1 (range=10-25),Warrior,0001A6BA,0001A6B9,50+(PC-1)×11.7,50,50+(PC-1)×3.3,"Heavy Armor, One-handed, Archery, Block",No Crime,Unaggressive,CrimeFactionPale; DawnstarBrinaHouseFaction; I...,CombatWarrior1H,Yes,NaN,None,NaN,NaN,None,None,NaN,No,No,None,No,No,NaN,No,No,NaN,Vanilla,10.0,155.3,50.0,79.7
3,https://en.uesp.net/wiki/Skyrim:Dawnstar_Guard,Dawnstar Guard,NaN,NaN,Nord,Male,PCx1 (Range=20-50),Warrior,NaN,0002D794,NaN,NaN,NaN,NaN,Indeterminate,Indeterminate,CrimeFactionPale; Stormcloaks; Sons of Skyrim;...,None,No,Dawnstar,None,NaN,NaN,None,None,NaN,No,No,None,No,No,NaN,No,No,NaN,Vanilla,20.0,NaN,NaN,NaN
4,https://en.uesp.net/wiki/Skyrim:Babette,Babette,NaN,NaN,Breton,Female,10,Assassin,0001D4BC,0001D4B7,125,15,95,"Archery, One-handed, Sneak, Conjuration, Light...",No Crime,Unaggressive,Alchemy Trainer; Babette's Bed Ownership; Babe...,TrainerAlchemyMaster,Yes,"Dark Brotherhood Sanctuary, Dawnstar Sanctuary",Alchemy (Master),True,500 (+1000 Master Trader),See Standard Merchandise,"Animal Parts, Food, Ingredients, Poisons, Poti...","Vampire, Undead, Child",No,No,None,No,No,NaN,No,No,NaN,Vanilla,10.0,125.0,15.0,95.0


#### Identifying a unique id

Check which columns we can use as a unique index. Perhaps name?

In [55]:
print(npcs.shape)
npcs.nunique()

(1009, 40)


Source             1009
Name               1000
Home City            32
House               142
Race                 16
Gender                4
Level               138
Class                61
RefID               975
BaseID             1003
Health              192
Magicka             122
Stamina             148
Primary Skills      187
Morality              5
Aggression            4
Faction(s)          731
Class Details        72
Essential            63
Location            292
Training             47
Merchant              1
Gold                 25
Sells                82
Buys                 20
Race Details          4
Follower              3
Respawns              4
Store                73
AdoptHF               3
Protected             4
Perks                68
Marry                 2
StewardHF             2
Horses                6
Added by             11
Level (PC=10)        38
Health (PC=10)      178
Magicka (PC=10)     111
Stamina (PC=10)     127
dtype: int64

In [57]:
npcs[npcs['Name'].isna()]

,Source,Name,Home City,House,Race,Gender,Level,Class,RefID,BaseID,Health,Magicka,Stamina,Primary Skills,Morality,Aggression,Faction(s),Class Details,Essential,Location,Training,Merchant,Gold,Sells,Buys,Race Details,Follower,Respawns,Store,AdoptHF,Protected,Perks,Marry,StewardHF,Horses,Added by,Level (PC=10),Health (PC=10),Magicka (PC=10),Stamina (PC=10)


It looks like some NPCs have the same name. This seems viable - for example, Ysgramor is the name given to a heroic Nord NPC, but also the name given to a dog.

In [77]:
name_counts = npcs.groupby('Name')['Source'].nunique()
display(name_counts[name_counts > 1])
display(npcs[npcs['Name']=='Ysgramor'])

Name
Barknar                  2
Emperor Titus Mede II    2
Eydis                    2
Gunjar                   2
Karita                   2
Mogrul                   2
Nikulas                  2
Thalmor                  2
Ysgramor                 2
Name: Source, dtype: int64

,Source,Name,Home City,House,Race,Gender,Level,Class,RefID,BaseID,Health,Magicka,Stamina,Primary Skills,Morality,Aggression,Faction(s),Class Details,Essential,Location,Training,Merchant,Gold,Sells,Buys,Race Details,Follower,Respawns,Store,AdoptHF,Protected,Perks,Marry,StewardHF,Horses,Added by,Level (PC=10),Health (PC=10),Magicka (PC=10),Stamina (PC=10)
504,https://en.uesp.net/wiki/Skyrim:Ysgramor_(dog),Ysgramor,Windhelm,Hollyfrost Farm,Animals,Indeterminate,2,Indeterminate,00023EEE,00023EED,21,0,24,Sneak,Indeterminate,Indeterminate,Creature Faction; DogFaction; Hollyfrost Farm ...,None,No,NaN,None,NaN,NaN,None,None,NaN,No,No,None,No,No,NaN,No,No,NaN,Vanilla,2.0,21.0,0.0,24.0
893,https://en.uesp.net/wiki/Skyrim:Ysgramor,Ysgramor,NaN,NaN,Nord,Male,50,Citizen,00098BD2,00098BD1,458,213,214,"Alchemy, Enchanting, Smithing, Archery, Light ...",No Crime,Aggressive,FavorExcludedFaction; MQ304SovngardeHeroFactio...,None,Yes,"Hall of Valor, Sovngarde",None,NaN,NaN,None,None,NaN,No,No,None,No,No,NaN,No,No,NaN,Vanilla,50.0,458.0,213.0,214.0


However it seems strange that there are *two* Emperor Titus Mede IIs... but it looks like they may be two separate characters (see below).

Doing a little bit of reading online reveals that the decoy version in the game is actually a different character impersonating Emperor Titus.

In [74]:
npcs.loc[npcs['Name']=='Emperor Titus Mede II', 'Source']
print(npcs.loc[npcs['Name']=='Emperor Titus Mede II', 'Source'][332])
print(npcs.loc[npcs['Name']=='Emperor Titus Mede II', 'Source'][878])

https://en.uesp.net/wiki/Skyrim:Emperor_Titus_Mede_II_(decoy)
https://en.uesp.net/wiki/Skyrim:Emperor_Titus_Mede_II_(real)


So we will use the original dataframe index as a unique id.

#### Processing to create useful feature columns

Some of the columns (Primary Skills, Faction(s), Location, Buys, Race Details) consist of a string with comma separated values. A first job would be to separate out these values and create a sparse matrix instead, which will be easier to feed into models.

In [79]:
def return_sparse_matrix(df, column):
    series = df[column]
    series_split = series.astype(str).apply(lambda x: x.split(', '))
    mlb = MultiLabelBinarizer() # Converts from series of lists to a binary matrix 
    return pd.DataFrame(mlb.fit_transform(series_split),columns=mlb.classes_, index=df.index)

In [80]:
primary_skills_df = return_sparse_matrix(npcs, 'Primary Skills')
factions_df = return_sparse_matrix(npcs, 'Faction(s)')
location_df = return_sparse_matrix(npcs, 'Location')
buys_df = return_sparse_matrix(npcs, 'Buys')
race_details_df = return_sparse_matrix(npcs, 'Race Details')

In [95]:
primary_skills_df

,Alchemy,Alteration,Archery,Block,Conjuration,Destruction,Enchanting,Heavy Armor,Illusion,Light Armor,Lockpicking,One-handed,Pickpocket,Restoration,Smithing,Sneak,Speech,Two-handed,nan
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0
1,0,0,1,1,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0
2,0,0,1,1,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
4,0,0,1,0,1,0,0,0,0,1,0,1,0,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1004,0,0,0,0,1,0,0,0,0,1,0,1,0,0,0,1,0,0,0
1005,0,1,0,1,0,0,0,1,0,1,0,1,0,1,0,0,0,1,0
1006,0,1,0,1,0,0,0,1,0,1,0,1,0,1,0,0,0,0,0
1007,0,1,0,0,0,1,0,0,0,0,0,0,0,1,0,1,0,0,0


In [82]:
race_details_df

,Child,Ghost,Undead,Vampire,nan
0,0,0,0,0,1
1,0,0,0,0,1
2,0,0,0,0,1
3,0,0,0,0,1
4,1,0,1,1,0
...,...,...,...,...,...
1004,0,0,1,1,0
1005,0,0,0,0,1
1006,0,0,0,0,1
1007,0,0,0,0,1


In [86]:
location_df.columns # Need to split on a regex with a capital letter for this?

Index(['A small camp west of Anga's Mill', 'Abandoned Building',
       'Abandoned Lodge', 'Abandoned Shack', 'Alftand Cathedral',
       'Alftand Glacial Ruins', 'Altar of Thrond', 'Angarvunde', 'Angi's Camp',
       'Anise's Cabin',
       ...
       'before the dragon attack', 'nan', 'or traveling between them',
       'outside of Fort Dawnguard', 'outside of Riften Fishery',
       'south of Hrodulf's House', 'then Druadach Redoubt',
       'then Sky Haven Temple', 'then Thirsk Mead Hall',
       'then relocates to The Ragged Flagon - Cistern'],
      dtype='object', length=302)

In [93]:
factions_df.columns

Index(['Actors dead bodies won't trigger World Interactions; CrimeFactionRift; FavorExcludedFaction; Riften Honorhall Orphanage Faction; TownRiftenFaction',
       'Actors in this will never fill world interaction aliases; Alteration Trainer; College of Winterhold 4(Wizard); CrimeFactionWinterhold; Skill Trainer; Tolfdir Services',
       'Actors in this will never fill world interaction aliases; CrimeFactionHaafingar; SolitudeOpeningPrisonerFaction; TownSolitudeFaction',
       'Actors in this will never fill world interaction aliases; DLC2SkaalDialogueFaction; DLC2SkaalVillageCitizenFaction; DLC2SVGreathallFaction',
       'Actors in this will never fill world interaction aliases; Skaal Shaman; DLC2SkaalDialogueFaction; DLC2SkaalVillageCitizenFaction',
       'Adril and Cindiri Arano's Shared Faction; DLC2CrimeRavenRockFaction; Raven Rock Morvayn Manor Faction',
       'Adventurers',
       'Alchemy Trainer; Babette's Bed Ownership; Babette's Dark Brotherhood Services; The Dark Broth

In [90]:
location_df.sum().sort_values(ascending=False).head(n=50)

nan                              374
Volkihar Keep                     16
College of Winterhold             15
Bloated Man's Grotto              13
Fort Dawnguard                    13
Jorrvaskr                         12
Sovngarde                         12
Dragonsreach                      11
Understone Keep                   11
Hall of Valor                     10
Candlehearth Hall                  9
Dark Brotherhood Sanctuary         9
Bards College                      9
Hall of Attainment                 9
The Ragged Flagon - Cistern        9
The Treasury House                 8
Cidhna Mine                        8
The Warrens                        8
Blue Palace                        8
Windpeak Inn                       7
Haelga's Bunkhouse                 7
then Druadach Redoubt              7
Mistveil Keep                      7
Thalmor Embassy                    6
Left Hand Mine                     6
Forgotten Vale                     6
Dayspring Canyon                   6
C

In [91]:
location_df.sum().sort_values(ascending=True).head(n=50)

A small camp west of Anga's Mill          1
Mzulft                                    1
Mzulft Aedrome                            1
Narfi's ruined house in Ivarstead         1
North of Ashfallow Citadel                1
Old Attius Farm                           1
Old Hroldan Inn                           1
On the southern tip of Solstheim          1
Outside of Moss Mother Cavern             1
Outside the Abandoned House               1
Pale Imperial Camp                        1
Pale Stormcloak Camp                      1
Mzinchaleft                               1
Pinewatch                                 1
Proudspire Manor                          1
Radiant                                   1
Raldbthar                                 1
Ramshackle Trading Post                   1
Random Encounters                         1
Random Forsworn camp                      1
Rannveig's Fast                           1
Reach Imperial Camp                       1
Reachcliff Cave (after personal 

### Analysis

#### Primary Skills correlations

In [97]:
npcs_plus = npcs.join(primary_skills_df)

In [98]:
npcs_plus.columns

Index(['Source', 'Name', 'Home City', 'House', 'Race', 'Gender', 'Level',
       'Class', 'RefID', 'BaseID', 'Health', 'Magicka', 'Stamina',
       'Primary Skills', 'Morality', 'Aggression', 'Faction(s)',
       'Class Details', 'Essential', 'Location', 'Training', 'Merchant',
       'Gold', 'Sells', 'Buys', 'Race Details', 'Follower', 'Respawns',
       'Store', 'AdoptHF', 'Protected', 'Perks', 'Marry', 'StewardHF',
       'Horses', 'Added by', 'Level (PC=10)', 'Health (PC=10)',
       'Magicka (PC=10)', 'Stamina (PC=10)', 'Alchemy', 'Alteration',
       'Archery', 'Block', 'Conjuration', 'Destruction', 'Enchanting',
       'Heavy Armor', 'Illusion', 'Light Armor', 'Lockpicking', 'One-handed',
       'Pickpocket', 'Restoration', 'Smithing', 'Sneak', 'Speech',
       'Two-handed', 'nan'],
      dtype='object')

In [99]:
npcs_plus.groupby('Race').sum()

,Level (PC=10),Health (PC=10),Magicka (PC=10),Stamina (PC=10),Alchemy,Alteration,Archery,Block,Conjuration,Destruction,Enchanting,Heavy Armor,Illusion,Light Armor,Lockpicking,One-handed,Pickpocket,Restoration,Smithing,Sneak,Speech,Two-handed,nan
Race,,,,,,,,,,,,,,,,,,,,,,,
Animals,49.0,811.000000,4.000,704.000000,0,0,3,3,0,0,0,0,0,0,0,3,0,0,0,9,0,0,1
Argonian,134.0,2217.800000,1358.200,1332.800000,1,1,4,2,0,2,1,1,1,6,10,9,3,0,1,12,4,0,1
Breton,731.0,10813.580000,7278.910,6116.450000,21,14,17,20,42,18,1,5,10,13,0,29,3,13,1,19,15,6,9
Dark Elf,929.0,15480.520000,8787.280,7436.160000,19,18,11,14,13,64,8,10,5,10,2,24,7,16,9,26,11,11,9
Dremora,4.0,85.000000,50.000,60.000000,0,0,1,1,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0
Elder,757.0,8969.530000,4081.900,5968.460000,18,7,13,6,13,7,12,4,4,8,1,15,2,13,15,14,16,5,10
High Elf,448.0,6626.900000,5286.400,2319.500000,0,15,3,2,15,24,1,3,20,2,0,6,0,17,0,6,1,1,3
Imperial,633.5,10430.220000,6064.120,6909.960000,3,6,17,15,5,13,10,13,2,12,1,31,3,57,5,14,9,9,9
Khajiit,179.0,2689.400000,1356.800,1885.200000,0,1,3,6,1,3,0,4,1,5,2,9,2,2,0,9,1,4,8
